# ELAN to GSheet and WAV
Rolando Coto-Solano (Rolando.A.Coto.Solano@dartmouth.edu)<br>
Dartmouth College. Last update: 20250601

The program takes two main inputs:

* `nameTabFile`: A file (TSV or TXT) exported from ELAN, with six tab-separated columns. This file needs to be in the `processed-elan-files` folder of the sandbox you will use. Column 3 of the file should have the beginning time of the annotation (in seconds). Column 4 should have the end time in seconds. Column 6 should have the annotation text. (ELAN puts the name of the tier in the first column, the second column is blank, and the fifth column is the duration of the annotation in seconds). This should be exported from a tier that contains a single speaker.<br>
* `bigWavFile`: The WAV, MP3 or MP4 file that has the annotations in the TSV file. It needs to be in the `processed-elan-files` folder of the sandbox you will use.

The program also takes the following inputs:

* `destinationSandbox`: The name of the sandbox you are using. The defaults are {sandbox-user and all-wavs}, but you can use whichever you specified during the installation.<br>
* `installationFolder`: The folder where the ASR sandboxes are contained. The default value is 2023-asr-workshop, but you should use the one you specified during the installation.<br>
* `speakerCode`: A short, two or three letter code to distinguish this speaker from other speakers in the dataset.<br>
* `prefixSmallAudioFiles`: A prefix to distinguish these audio files from others in your complete dataset. In most cases it'll be the same as the wav file name (minus the extension).<br>
* `speakerGender`: The older Kaldi system separates high from low pitch inputs. It does it by distinguishing them with {m,f}. The newer system does not need this information.

The program takes the information from the TSV and WAV, (1) splits the WAV into smaller pieces and puts them in the {sandbox}/wav folder, and (2) adds the TSV information to the Google Spreadsheet that corresponds to the sandbox.

## Fill out metadata and connect to Google Drive

In [68]:
# Fill out the metadata for the file you want to process

destinationSandbox = "sandbox-user"                                           # Please type sandbox-user or all-wavs (or one of your user folders)
installationFolder = "202407-asr-tothesis"

nameTabFile = "ali_kc_ṣobbiḍ_by_adrian_rios.txt"                                               # The file needs to be in the processed-elan-files folder of your sandbox.
speakerCode = "akṣ-ar"
bigWavFile = "ali_kc_ṣobbiḍ_by_adrian_rios.wav"                                                    # The file needs to be in the processed-elan-files folder of your sandbox.
prefixSmallAudioFiles = "AR"                                         # Prefix for the split audio files
speakerGender = "F"                                                           # Kaldi only has binary m/f. Wav2Vec2 doesn't care

urlSandbox = "https://docs.google.com/spreadsheets/d/1L3U6fU7azI39Qla9_aD12RDB4LobW67EGyed-SKd6WM/edit?usp=sharing"

## Prepare libraries and access to Google Drive

In [69]:
# Load relevant libraries
import math
import pandas as pd
from datetime import date
import os
import wave
import contextlib
from google.colab import drive

# It needs this permission to access the ASR spreadsheets in your GDrive
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [70]:
# It needs this permission to read and write ASR files into your GDrive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


## File processing

In [71]:
#==============================================
# Read in sandbox information
#==============================================

foldersToFindFiles = "/content/drive/MyDrive/" + installationFolder + "/" + destinationSandbox + "/processed-elan-files/"
folderWhereWavsAreStored = "/content/drive/MyDrive/" + installationFolder + "/" + destinationSandbox + "/wav/"
gsheetURL = urlSandbox

In [72]:
#==============================================
# Support functions
#==============================================

def countDigits(number):
  count=0
  while(number>0):
    count=count+1
    number=number//10
  return(count)

def addZerosToNumber(number, maxNumber):

  digitsMax = countDigits(maxNumber)
  digitsNumber = countDigits(number)

  zerosToAdd = digitsMax - digitsNumber
  zeros = ""

  for i in range(0,zerosToAdd):
    zeros += "0"

  retNum = zeros + str(number)
  return(retNum)

def reformatTranscription(input):

  output = input

  # Modify this function if you need to do extra editing
  # of your ELAN transcription (e.g. replacing special
  # characters or eliminating punctuation)

  punctuation = [ "[", "]", "\"", "(", ")", ".", "\u0f7b", "_", "|", "》", "?", "!", "/", ',', '-', '?', '<', '…', '>' ]

  for p in punctuation: output = output.replace(p, " ")
  for i in range(0,5): output = output.replace("  "," ")
  output = output.strip().lower()

  return(output)

In [73]:
#==============================================
# Read in tab-separated file
#==============================================

transcriptionColumn = 5   # It's usually 5

tabFile = open(foldersToFindFiles + nameTabFile, 'r')
tabFile = tabFile.readlines()

timeStart = []
timeEnd = []
transcriptions = []

for line in tabFile:

  line = line.replace("\n","")
  lineSplit = line.split("\t")
  print(lineSplit)

  start = float(lineSplit[2])
  end = float(lineSplit[3])
  transcription = lineSplit[transcriptionColumn]
  duration = end - start

  if (duration > 0 and reformatTranscription(transcription) != ""):
    timeStart.append(lineSplit[2])
    timeEnd.append(lineSplit[3])
    transcriptions.append(lineSplit[transcriptionColumn])

print("I found " + str(len(transcriptions)) + " valid and non-empty annotations in the file " + nameTabFile)

['Adrian Rios', '', '1.06', '2.28', '1.22', 'ali kc ṣobbiḍ']
['Adrian Rios', '', '3.16', '5.49', '2.33', 'o’odham ñiokĭculda o’ohona']
['Adrian Rios', '', '7.43', '8.71', '1.28', 'ña:a g ’ali']
['Adrian Rios', '', '11.49', '12.8', '1.31', 'ñeid g ’ali']
['Adrian Rios', '', '13.41', '14.6', '1.19', 'ña:a g ṣobbiḍ']
['Adrian Rios', '', '15.41', '16.57', '1.16', 'ñeid g ṣobbiḍ']
['Adrian Rios', '', '17.23', '19.32', '2.09', 'ña:a g ’ali kc ṣobbiḍ']
['Adrian Rios', '', '19.58', '21.52', '1.94', 'ñeid g ’ali kc ṣobbiḍ']
['Adrian Rios', '', '22.37', '24.35', '1.98', 'ñeid g ’ali mo cicwi']
['Adrian Rios', '', '24.6', '28.129', '3.529', 'ñeid g ’ali mo cicwi g ṣobbiḍ']
I found 10 valid and non-empty annotations in the file ali_kc_ṣobbiḍ_by_adrian_rios.txt


### Split audio files

In [74]:
#============================================================
# If the audio is in an MP3 file, convert it to a .wav file
#============================================================

if (".mp3" in bigWavFile):
  newaudioFile = bigWavFile.replace(".mp3", ".wav")
  command = "ffmpeg -y -i "
  command += foldersToFindFiles + bigWavFile
  command += " -acodec pcm_u8 -ar 22050 "
  command += foldersToFindFiles + newaudioFile

  !$command
  bigWavFile = newaudioFile

In [75]:
#============================================================
# Separate the big file into smaller wav files
#============================================================

filenames = []
print(len(timeStart))


points = []
for start, end in zip(timeStart, timeEnd):
  i = len(points)-1
  tempName = speakerCode + "-" + prefixSmallAudioFiles + "-" + addZerosToNumber(i+1,len(timeStart)) + ".wav"
  tempName2 = speakerCode + "-" + prefixSmallAudioFiles + "-" + addZerosToNumber(i+2,len(timeStart)) + ".wav"
  if points and points[-1] == start:
    points.append(end)
    filenames.append(tempName)
  else:
    points.append(start)
    points.append(end)
    filenames.append(tempName2)

pointSeq = ','.join(str(t) for t in points)
inFileName = foldersToFindFiles + bigWavFile
zeros = countDigits(len(timeStart))
outFileName = folderWhereWavsAreStored + speakerCode + '-' + prefixSmallAudioFiles + f'-%0{zeros}d.wav'
!ffmpeg -y -i "$inFileName" -f segment -ac 1 -ar 16000 -async 1 -segment_times $pointSeq $outFileName


10
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --e

### Add metadata and transcriptions to Google Sheet

In [76]:
#==================================================================
# Read GSheet as Pandas, add new rows, and rewrite the
# sandbox GSheet with the new Pandas dataframe.
#==================================================================

from gspread_dataframe import set_with_dataframe

wb = gc.open_by_url(gsheetURL)
sheet = wb.worksheet('wav-metadata')
rows = sheet.get_all_values()
sandboxRows = pd.DataFrame(rows)

rowsAtStart = len(sandboxRows.index)

today = date.today()
todaysDate = today.strftime("%Y%m%d")

for i in range(0,len(timeStart)):
  with contextlib.closing(wave.open(folderWhereWavsAreStored + filenames[i],'r')) as f:
      frames = f.getnframes()
      rate = f.getframerate()
      duration = frames / float(rate)
  inValues = [filenames[i], destinationSandbox.replace("sandbox-",""), todaysDate, speakerCode, speakerGender, os.path.getsize(folderWhereWavsAreStored + filenames[i]), duration, "", "", reformatTranscription(transcriptions[i]), transcriptions[i]]
  sandboxRows.loc[len(sandboxRows)] = inValues

set_with_dataframe(sheet, sandboxRows)  # Write all rows to sandbox GSheet
sheet.delete_rows(1,1)                     # Erase the pandas numerical headers
rowsAtEnd = len(sandboxRows.index)

print("Total rows before new data: " + str(rowsAtStart))
print("Total rows after new data:  " + str(rowsAtEnd))
print("Added rows:                 " + str(rowsAtEnd-rowsAtStart))

Total rows before new data: 1234
Total rows after new data:  1244
Added rows:                 10
